# Type 1
##### Max: 2 overlapping groups
##### e.g. [0, 8), [5, 10)

In [1]:
import numpy as np
import sys
np.set_printoptions(threshold=sys.maxsize)
import pandas as pd
import scipy as sp

import jax
import jax.numpy as jnp

import altair as alt
SEED = 123

In [2]:
x = np.linspace(0, 100, 100)
log_exposure = np.exp(-x / 10)
f = 0.5 * np.sin(x / 20) + 0.5 * np.cos(x / 10)
rate = np.exp(log_exposure + f)

np.random.seed(SEED) ; y = np.random.poisson(rate, size=(100,))

df_data = pd.DataFrame({
	'x': x,
	'log_exposure': log_exposure,
	'f': f,
	'rate': rate,
	'y': y
})

print(f"Sum of y: {df_data['y'].sum()}")
df_data.head()

Sum of y: 152


,x,log_exposure,f,rate,y
0,0.000000,1.000000,0.500000,4.481689,5
1,1.010101,0.903924,0.522693,4.164587,6
2,2.020202,0.817078,0.540251,3.885801,2
3,3.030303,0.738577,0.552686,3.637378,2
4,4.040404,0.667617,0.560064,3.413307,6


In [3]:
def expand_data(df, y_col):
    result = pd.DataFrame(columns=df.columns)

    for row_id, repeat in zip(range(df.shape[0]), df[y_col]):
        row = df.loc[[row_id]]
        if int(repeat) == 0:
            result = pd.concat([result, row], ignore_index=True)
            continue
        duplicated_rows = pd.concat([row] * repeat, ignore_index=True)
        result = pd.concat([result, duplicated_rows], ignore_index=True)

    return result

In [4]:
expanded_df = expand_data(df_data, 'y')
expanded_df.tail()

/var/folders/r0/qp6_p2111v1dzb0fg2bbfqsr0000gn/T/ipykernel_10688/1639621420.py:10: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  result = pd.concat([result, duplicated_rows], ignore_index=True)


,x,log_exposure,f,rate,y
174,95.959596,0.000068,-0.990862,0.371282,0
175,96.969697,0.000061,-0.976969,0.376474,1
176,97.979798,0.000056,-0.956903,0.384102,1
177,98.989899,0.000050,-0.930837,0.394243,0
178,100.000000,0.000045,-0.898998,0.406996,0


In [5]:
rate = (
  alt.Chart(df_data)
  .mark_line(color='red')
  .encode(
		x=alt.X('x', title='x'),
		y=alt.Y('rate', title='Rate'),
	)
)

count = alt.Chart(df_data).mark_point().encode(
	x=alt.X('x', title='x'),
	y=alt.Y('y', title='Count'),
)

(rate + count).properties(width=600)


alt.LayerChart(...)

In [6]:
def assign_bin(x, y, bins, seed=123):
    np.random.seed(seed)
    cum_bins_id = []
    for age, count in zip(x, y):
        temp_bins = []
        for bin_id, (lower, upper) in enumerate(bins):
            if lower <= age < upper:
                temp_bins.append(bin_id)
        if not temp_bins:
            if count == 0:
                cum_bins_id.append(pd.NA) 
            else:
                cum_bins_id.extend([pd.NA] * count)
        else:
            if count == 0:
                cum_bins_id.append(np.random.choice(temp_bins))
            else:
                cum_bins_id.extend(np.random.choice(temp_bins, size=count))
    cum_bins = []
    for bin_id in cum_bins_id:
        if pd.isna(bin_id):
            cum_bins.append(pd.NA)
        else:
            cum_bins.append(bins[bin_id])
    return cum_bins

In [7]:
type_1_bins = [(0, 10), (5,15), (10, 20), (25,35), (20, 30), (35, 45), (30, 40), (45, 55), (40, 50), (55, 65), (50, 60), (60, 70), (70, 80), (80, 90), (90, 100.1)]
expanded_df["bin"] = assign_bin(x, y, type_1_bins)
expanded_df.head()

,x,log_exposure,f,rate,y,bin
0,0.0,1.0,0.5,4.481689,5,"(0, 10)"
1,0.0,1.0,0.5,4.481689,5,"(0, 10)"
2,0.0,1.0,0.5,4.481689,5,"(0, 10)"
3,0.0,1.0,0.5,4.481689,5,"(0, 10)"
4,0.0,1.0,0.5,4.481689,5,"(0, 10)"


In [8]:
# Keeping unique rows
expanded_df = expanded_df.drop_duplicates(subset=['x', 'log_exposure', 'f', 'rate', 'y', 'bin']).reset_index(drop=True)
expanded_df['y'] = np.asarray(expanded_df['y'].values, dtype=np.int64)

# converting 'bin' to an interval closed on left
expanded_df['bin'] = pd.IntervalIndex.from_arrays(
    [interval[0] for interval in expanded_df['bin']],
    [interval[1] for interval in expanded_df['bin']],
    closed='left'
)

expanded_df['bin'] = pd.Categorical(expanded_df['bin'])

print(expanded_df.shape)
expanded_df.head(15)

(118, 6)


,x,log_exposure,f,rate,y,bin
0,0.000000,1.000000,0.500000,4.481689,5,"[0.0, 10.0)"
1,1.010101,0.903924,0.522693,4.164587,6,"[0.0, 10.0)"
2,2.020202,0.817078,0.540251,3.885801,2,"[0.0, 10.0)"
3,3.030303,0.738577,0.552686,3.637378,2,"[0.0, 10.0)"
4,4.040404,0.667617,0.560064,3.413307,6,"[0.0, 10.0)"
5,5.050505,0.603475,0.562500,3.209050,2,"[0.0, 10.0)"
6,5.050505,0.603475,0.562500,3.209050,2,"[5.0, 15.0)"
7,6.060606,0.545496,0.560156,3.021193,2,"[0.0, 10.0)"
8,7.070707,0.493086,0.553242,2.847179,3,"[0.0, 10.0)"
9,8.080808,0.445713,0.542012,2.685118,5,"[5.0, 15.0)"


In [9]:
# y_agg: the number of counts in each age range
# N: the number of non-missing values in each age range
tmp = expanded_df.groupby('bin', observed=True).agg(
	y_agg = ('y', 'sum'),
	N = ('y', 'count')
).reset_index()

y_agg = np.asarray(tmp['y_agg'].values, dtype=np.int64)
tmp['y_agg'] = y_agg
tmp

,bin,y_agg,N
0,"[0.0, 10.0)",36,10
1,"[5.0, 15.0)",23,7
2,"[10.0, 20.0)",18,8
3,"[20.0, 30.0)",12,8
4,"[25.0, 35.0)",7,6
5,"[30.0, 40.0)",5,6
6,"[35.0, 45.0)",11,6
7,"[40.0, 50.0)",14,6
8,"[45.0, 55.0)",18,9
9,"[50.0, 60.0)",16,7


In [10]:
# rate_agg: the avg number of counts per non-missing age associated with each age range
expanded_df = expanded_df.merge(tmp, on='bin', how='left')
expanded_df['rate_agg'] = expanded_df['y_agg'] / expanded_df['N']
expanded_df['rate_agg'] = np.asarray(expanded_df['rate_agg'].values, dtype=np.float64)
expanded_df.head(10)

,x,log_exposure,f,rate,y,bin,y_agg,N,rate_agg
0,0.000000,1.000000,0.500000,4.481689,5,"[0.0, 10.0)",36,10,3.600000
1,1.010101,0.903924,0.522693,4.164587,6,"[0.0, 10.0)",36,10,3.600000
2,2.020202,0.817078,0.540251,3.885801,2,"[0.0, 10.0)",36,10,3.600000
3,3.030303,0.738577,0.552686,3.637378,2,"[0.0, 10.0)",36,10,3.600000
4,4.040404,0.667617,0.560064,3.413307,6,"[0.0, 10.0)",36,10,3.600000
5,5.050505,0.603475,0.562500,3.209050,2,"[0.0, 10.0)",36,10,3.600000
6,5.050505,0.603475,0.562500,3.209050,2,"[5.0, 15.0)",23,7,3.285714
7,6.060606,0.545496,0.560156,3.021193,2,"[0.0, 10.0)",36,10,3.600000
8,7.070707,0.493086,0.553242,2.847179,3,"[0.0, 10.0)",36,10,3.600000
9,8.080808,0.445713,0.542012,2.685118,5,"[5.0, 15.0)",23,7,3.285714


In [24]:
# Create a clean dataset without the Interval objects for plotting
df_plot = expanded_df.drop(columns=['bin'])

rate = (
  alt.Chart(df_plot)
  .mark_line(color='red')
  .encode(
		x=alt.X('x', title='x'),
		y=alt.Y('rate', title='Rate'),
	)
)

count = alt.Chart(df_plot).mark_point().encode(
	x=alt.X('x', title='x'),
	y=alt.Y('y', title='Count'),
)

rate_agg = (
  alt.Chart(df_plot)
  .mark_line(color='blue')
  .encode(
		x=alt.X('x', title='x'),
		y=alt.Y('rate_agg', title='Aggregated Rate'),
	)
)

(rate_agg + rate + count).properties(width=600)

alt.LayerChart(...)

In [12]:
import numpyro
from numpyro.contrib.hsgp.approximation import hsgp_squared_exponential
from numpyro.contrib.hsgp.laplacian import eigenfunctions
from numpyro.contrib.hsgp.spectral_densities import (
	diag_spectral_density_squared_exponential
)
import numpyro.distributions as dist
from numpyro.infer import SVI, MCMC, NUTS, Predictive

In [13]:
def create_interval_index_array(intervals_series, grid_start=0, grid_end=None, grid_step=1):
    """
    Create an index array that maps intervals to points on a fine integer grid.
    
    Parameters:
    -----------
    intervals_series : pandas.Series
        A pandas Series of categorical dtype containing pd.Interval objects
        that are closed on the left and open on the right [left, right).
    grid_start : int, default 0
        The starting point of the integer grid.
    grid_end : int, optional
        The ending point of the integer grid. If None, inferred from intervals.
    grid_step : int, default 1
        The step size for the integer grid.
    
    Returns:
    --------
    numpy.ndarray
        An index array where each element corresponds to a grid point and
        contains the index of the interval that contains that grid point.
        Points not in any interval get index -1.
    
    Example:
    --------
    >>> import pandas as pd
    >>> import numpy as np
    >>> 
    >>> # Create some intervals
    >>> x = np.linspace(0, 10, 11)
    >>> intervals = pd.cut(x, bins=3, right=False)
    >>> 
    >>> # Create index array for fine grid from 0 to 10
    >>> index_array = create_interval_index_array(intervals, 0, 10)
    >>> print(index_array)
    """
    
    # Get unique intervals and their categorical codes
    unique_intervals = intervals_series.cat.categories
    interval_codes = intervals_series.cat.codes
    
    # Determine grid bounds if not provided
    if grid_end is None:
        max_right = max(interval.right for interval in unique_intervals)
        grid_end = int(np.floor(max_right))

    # Create the fine integer grid
    grid_points = np.arange(grid_start, grid_end + grid_step, grid_step)
    
    # Initialize index array with -1 (indicating no interval contains the point)
    index_array = np.full(len(grid_points), -1, dtype=int)
    
    # For each unique interval, find which grid points it contains
    for i, interval in enumerate(unique_intervals):
        # Find grid points that fall within this interval [left, right)
        mask = (grid_points >= interval.left) & (grid_points < interval.right)

        
        
        # Set the index array to the categorical code for this interval
        index_array[mask] = i
    
    return index_array


def create_mapping_matrix(intervals_series, grid_start=0, grid_end=None, grid_step=1):
    """
    Create a mapping matrix that aggregates values from a fine grid to intervals.
    
    Parameters:
    -----------
    intervals_series : pandas.Series
        A pandas Series of categorical dtype containing pd.Interval objects.
    grid_start : int, default 0
        The starting point of the integer grid.
    grid_end : int, optional
        The ending point of the integer grid. If None, inferred from intervals.
    grid_step : int, default 1
        The step size for the integer grid.
    
    Returns:
    --------
    numpy.ndarray
        A matrix of shape (n_intervals, n_grid_points) where each row corresponds
        to an interval and contains 1s for grid points within that interval.
        This can be used for aggregation: aggregated_values = matrix @ grid_values
    """
    
    # Get the index array
    index_array = create_interval_index_array(intervals_series, grid_start, grid_end, grid_step)
    
    # Get number of unique intervals
    n_intervals = len(intervals_series.cat.categories)
    n_grid_points = len(index_array)
    
    # Create mapping matrix
    mapping_matrix = np.zeros((n_intervals, n_grid_points), dtype=int)
    
    # Fill the mapping matrix
    for grid_idx, interval_idx in enumerate(index_array):
        if interval_idx >= 0:  # Valid interval index
            mapping_matrix[interval_idx, grid_idx] = 1
    
    return mapping_matrix

In [14]:
def create_interval_index_array(intervals_series, grid_start=0, grid_end=None, grid_step=1):
    """
    Create an index array that maps intervals to points on a fine integer grid.
    
    Parameters:
    -----------
    intervals_series : pandas.Series
        A pandas Series of categorical dtype containing pd.Interval objects
        that are closed on the left and open on the right [left, right).
    grid_start : int, default 0
        The starting point of the integer grid.
    grid_end : int, optional
        The ending point of the integer grid. If None, inferred from intervals.
    grid_step : int, default 1
        The step size for the integer grid.
    
    Returns:
    --------
    numpy.ndarray
        An index array where each element corresponds to a grid point and
        contains the index of the interval that contains that grid point.
        Points not in any interval get index -1.
    
    Example:
    --------
    >>> import pandas as pd
    >>> import numpy as np
    >>> 
    >>> # Create some intervals
    >>> x = np.linspace(0, 10, 11)
    >>> intervals = pd.cut(x, bins=3, right=False)
    >>> 
    >>> # Create index array for fine grid from 0 to 10
    >>> index_array = create_interval_index_array(intervals, 0, 10)
    >>> print(index_array)
    """
       # Get unique intervals and their categorical codes
    unique_intervals = intervals_series.cat.categories
    interval_codes = intervals_series.cat.codes
    
    # Determine grid bounds if not provided
    if grid_end is None:
        max_right = max(interval.right for interval in unique_intervals)
        grid_end = int(np.floor(max_right))

    # Create the fine integer grid
    grid_points = np.arange(grid_start, grid_end + grid_step, grid_step)
    
    # Initialize index array with -1 (indicating no interval contains the point)
    index_array = np.full(len(grid_points), -1, dtype=int)
    
    # For each unique interval, find which grid points it contains
    for i, interval in enumerate(unique_intervals):
        # Find grid points that fall within this interval [left, right)
        mask = (grid_points >= interval.left) & (grid_points < interval.right)

        
        
        # Set the index array to the categorical code for this interval
        index_array[mask] = i
    
    return index_array
    

In [15]:
create_interval_index_array(expanded_df['bin'],
  grid_start=0,
  grid_end=99,
  grid_step=1)

# create_mapping_matrix(expanded_df['bin'],
#   grid_start=0,
#   grid_end=99,
#   grid_step=1)

array([ 0,  0,  0,  0,  0,  1,  1,  1,  1,  1,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  2,  3,  3,  3,  3,  3,  4,  4,  4,  4,  4,  5,  5,  5,  5,
        5,  6,  6,  6,  6,  6,  7,  7,  7,  7,  7,  8,  8,  8,  8,  8,  9,
        9,  9,  9,  9, 10, 10, 10, 10, 10, 11, 11, 11, 11, 11, 11, 11, 11,
       11, 11, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 13, 13, 13, 13, 13,
       13, 13, 13, 13, 13, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14])

In [16]:
# Standardize x
x_std = ((expanded_df['x'] - np.mean(expanded_df['x'])) / expanded_df['x'].std()).to_numpy()

mapping_matrix = create_mapping_matrix(
  expanded_df['bin'],
  grid_start=0,
  grid_end=99,
  grid_step=1
)

In [17]:
def model(x, log_exposure, N, map_matrix, L, M, non_centered, y=None):
  # --- Priors ---
  beta = numpyro.sample('baseline', dist.Normal(0, 10))
  sigma = numpyro.sample('sigma', dist.InverseGamma(5, 5))
  lenscale = numpyro.sample('lenscale', dist.InverseGamma(5, 5))
  
  # --- Parameterization ---
  f = hsgp_squared_exponential(
		x=x,
		alpha=sigma,
		length=lenscale,
		ell=L,
		m=M,
		non_centered=non_centered
	)
  
  # --- Likelihood ---
  log_rate = log_exposure + (beta + f)
  rate = numpyro.deterministic('rate', jnp.exp(log_rate))
  mu_agg = (map_matrix @ rate)
  numpyro.sample('y', dist.Poisson(mu_agg), obs=y)

In [18]:
df_train = expanded_df.groupby('bin').agg(
	y_agg=('y', 'sum'),
	N=('y', 'count')
).reset_index()

y_agg = df_train['y_agg'].values
N = df_train['N'].values

/var/folders/r0/qp6_p2111v1dzb0fg2bbfqsr0000gn/T/ipykernel_10688/668760269.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_train = expanded_df.groupby('bin').agg(


In [19]:
log_exposure=expanded_df['log_exposure'].values

In [20]:
print(y_agg.shape)
print(mapping_matrix.shape)
print(x_std.shape, expanded_df['log_exposure'].values.shape)
print(N)
print(df_train.shape)

(15,)
(15, 100)
(118,) (118,)
[10  7  8  8  6  6  6  6  9  7  6  9 10 10 10]
(15, 3)


In [21]:
print(expanded_df['bin'].shape)
print(expanded_df['bin'].isna().sum())

(118,)
0


In [22]:
print(f"Type of x: {type(x)}")
print(f"Type of log_exposure: {type(log_exposure)}")
print(f"Type of N: {type(N)}")
print(f"Type of map_matrix: {type(mapping_matrix)}")
print(f"Shape of map_matrix: {mapping_matrix.shape}")
print(f"Type of y_agg: {type(y_agg)}")
print(f"Type of x_std: {type(x_std)}")

print(df_train.shape)
print(N)

Type of x: <class 'numpy.ndarray'>
Type of log_exposure: <class 'numpy.ndarray'>
Type of N: <class 'numpy.ndarray'>
Type of map_matrix: <class 'numpy.ndarray'>
Shape of map_matrix: (15, 100)
Type of y_agg: <class 'numpy.ndarray'>
Type of x_std: <class 'numpy.ndarray'>
(15, 3)
[10  7  8  8  6  6  6  6  9  7  6  9 10 10 10]


In [23]:
sampler = NUTS(model)
mcmc = MCMC(sampler, num_warmup=500, num_samples=1_000, num_chains=4)

rng_key, rng_subkey = jax.random.split(jax.random.PRNGKey(42))

L = 2.0
M = 30
non_centered = True

mcmc.run(rng_subkey, 
	x=x_std, 
	log_exposure=log_exposure, 
	N=N, 
	map_matrix=mapping_matrix, 
	L=L, 
	M=M, 
	non_centered=non_centered, 
	y=y_agg
)

/var/folders/r0/qp6_p2111v1dzb0fg2bbfqsr0000gn/T/ipykernel_10688/2381358629.py:2: UserWarning: There are not enough devices to run parallel chains: expected 4 but got 1. Chains will be drawn sequentially. If you are running MCMC in CPU, consider using `numpyro.set_host_device_count(4)` at the beginning of your program. You can double-check how many devices are available in your system using `jax.local_device_count()`.
  mcmc = MCMC(sampler, num_warmup=500, num_samples=1_000, num_chains=4)


TypeError: dot_general requires contracting dimensions to have the same shape, got (100,) and (118,).

In [ ]:
import arviz as az
idata = az.from_numpyro(mcmc)

In [ ]:
df_sum = az.summary(
	data=idata,
	var_names=['rate']
)
df_sum

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
rate[0],7.631,1.351,5.267,10.231,0.044,0.026,894.0,1500.0,1.01
rate[1],7.074,1.148,5.105,9.343,0.031,0.022,1306.0,1670.0,1.00
rate[2],6.553,0.994,4.665,8.342,0.024,0.019,1680.0,1606.0,1.00
rate[3],6.060,0.877,4.498,7.760,0.020,0.017,1865.0,1580.0,1.00
rate[4],5.590,0.787,4.048,6.973,0.019,0.014,1757.0,1802.0,1.00
...,...,...,...,...,...,...,...,...,...
rate[95],0.422,0.171,0.134,0.738,0.008,0.004,490.0,967.0,1.00
rate[96],0.451,0.183,0.152,0.796,0.008,0.004,479.0,859.0,1.01
rate[97],0.487,0.198,0.181,0.877,0.009,0.005,468.0,878.0,1.01
rate[98],0.532,0.216,0.179,0.934,0.010,0.006,458.0,790.0,1.01


In [ ]:
df_plot['rate_est'] = df_sum['mean'].values

rate = (
  alt.Chart(df_plot)
  .mark_line(color='red')
  .encode(
		x=alt.X('x', title='x'),
		y=alt.Y('rate', title='Rate'),
	)
)

rate_est = (
  alt.Chart(df_plot)
	.mark_line(color='green', strokeDash=[5, 5])  # Changed
	.encode(
		x=alt.X('x', title='x'),
		y=alt.Y('rate_est', title='Estimated Rate'),
	)
)

count = alt.Chart(df_plot).mark_point().encode(
	x=alt.X('x', title='x'),
	y=alt.Y('y', title='Count'),
)

rate_agg = (
  alt.Chart(df_plot)
  .mark_line(color='blue', strokeDash=[5, 5])  # Changed color and added dashed line to distinguish
  .encode(
		x=alt.X('x', title='x'),
		y=alt.Y('rate_agg', title='Aggregated Rate'),
	)
)

(rate_agg + rate + count + rate_est).properties(width=600)

ValueError: Length of values (100) does not match length of index (118)